<a href="https://colab.research.google.com/github/Songhyunjeong/thingplug-lora-starter-kit/blob/master/%EC%82%AC%EB%B6%80%EC%9E%91%EC%82%AC%EB%B6%80%EC%9E%91_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

코랩에서 아래 라이브러리 설치


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00


In [4]:
!wget https://storage.googleapis.com/chrome-for-testing-public/129.0.6668.58/linux64/chromedriver-linux64.zip
!unzip /content/chromedriver-linux64.zip
!chmod +x '/content/chromedriver-linux64/chromedriver'
!mv '/content/chromedriver-linux64/chromedriver' /usr/local/bin/

--2024-10-03 12:50:05--  https://storage.googleapis.com/chrome-for-testing-public/129.0.6668.58/linux64/chromedriver-linux64.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.98.207, 74.125.139.207, 142.251.107.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.98.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9410841 (9.0M) [application/zip]
Saving to: ‘chromedriver-linux64.zip’

chromedriver-linux6 100%[===================>]   8.97M  --.-KB/s    in 0.1s    

2024-10-03 12:50:05 (93.3 MB/s) - ‘chromedriver-linux64.zip’ saved [9410841/9410841]

Archive:  /content/chromedriver-linux64.zip
  inflating: chromedriver-linux64/LICENSE.chromedriver  
  inflating: chromedriver-linux64/THIRD_PARTY_NOTICES.chromedriver  
  inflating: chromedriver-linux64/chromedriver  


In [5]:
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get -f install -y

--2024-10-03 12:50:10--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 173.194.216.91, 173.194.216.93, 173.194.216.190, ...
Connecting to dl.google.com (dl.google.com)|173.194.216.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 111878140 (107M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb’

google-chrome-stabl 100%[===================>] 106.69M   279MB/s    in 0.4s    

2024-10-03 12:50:10 (279 MB/s) - ‘google-chrome-stable_current_amd64.deb’ saved [111878140/111878140]

Selecting previously unselected package google-chrome-stable.
(Reading database ... 123620 files and directories currently installed.)
Preparing to unpack google-chrome-stable_current_amd64.deb ...
Unpacking google-chrome-stable (129.0.6668.89-1) ...
dpkg: dependency problems prevent configuration of google-chrome-stable:
 google-chrome-stable depends on libvulkan1; however:
  Package

In [ ]:
!chromium-browser --version
!chromedriver --version

/bin/bash: line 1: chromium-browser: command not found
ChromeDriver 129.0.6668.58 (81a06fb873a9b386848719cf9f93e59579fb5d4b-refs/branch-heads/6668@{#1318})


In [ ]:
!google-chrome --version

Google Chrome 129.0.6668.89 


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.binary_location = '/usr/bin/google-chrome'  # Chrome의 경로 명시
chrome_options.add_argument('--headless')  # 브라우저를 백그라운드에서 실행
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--remote-debugging-port=9222')

# Chromedriver 경로 설정
service = Service('/usr/local/bin/chromedriver')

# WebDriver 실행
driver = webdriver.Chrome(service=service, options=chrome_options)

# Google 페이지 열기
driver.get("https://www.naver.com")

# 페이지 제목 출력
print(driver.title)

# WebDriver 종료
#driver.quit()

NAVER


네이버 검색창에서 사업명 키워드로 검색(거래글 제외, 검색기간 : 2022.5.10.이후 부터 현재까지, 검색대상은 카페로 한정)

전체 게시글을 확인하여 12로 나눠주고 최종 페이지를 확인한다

크롤링(게시글 제목,내용,링크주소) 후 'crawled_posts.csv'로 저장

전체페이지 또는 특정페이지 크롤링하기(start_page,end_page 설정)


In [7]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchFrameException, NoSuchElementException

# Initialize the Selenium driver function
def initialize_driver():
    chrome_options = Options()
    chrome_options.binary_location = '/usr/bin/google-chrome'  # Chrome path
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--headless")  # Comment this line if you want to see the browser
    driver = webdriver.Chrome(service=Service('/usr/local/bin/chromedriver'), options=chrome_options)
    return driver

# Start the driver
driver = initialize_driver()

# Log in to Naver
driver.get("https://nid.naver.com/nidlogin.login")
time.sleep(2)
driver.find_element(By.ID, "id").send_keys("postpia")  # Replace with your Naver ID
driver.find_element(By.ID, "pw").send_keys("na700925!shj")  # Replace with your Naver password
driver.find_element(By.CLASS_NAME, "btn_login").click()
time.sleep(5)  # Wait for login to complete


# 데이터를 저장할 리스트들 초기화
all_titles = []
all_contents = []
all_links = []
seen_links = set()  # 중복 방지를 위한 링크 저장

# 현재 페이지의 게시글을 크롤링하는 함수
def crawl_current_page():
    # 현재 페이지의 HTML 소스를 가져옴
    req = driver.page_source
    soup = BeautifulSoup(req, 'html.parser')

    # 게시글 항목을 선택
    articles = soup.select("div.ArticleItem")

    print(f"Found {len(articles)} articles on the current page.")

    # 각 게시글의 제목, 본문, 링크를 추출하여 저장
    for article in articles:
        # 게시글 링크 추출
        link_element = article.find('a', href=True)
        article_link = link_element['href'] if link_element else "No link found"

        # 제목 추출
        title_element = article.select_one('strong.title')
        article_title = title_element.get_text(strip=True) if title_element else "No title found"

        # 본문 내용 추출
        content_element = article.select_one('p.text')
        article_content = content_element.get_text(strip=True) if content_element else "No content found"

        # 리스트에 추출된 데이터를 저장
        all_titles.append(article_title)
        all_contents.append(article_content)
        all_links.append(article_link)

        #rint(f'Title: {article_title}')
        #print(f'Content: {article_content[:200]}...')  # 본문 내용 일부 출력
        #print(f'Link: {article_link}')

# 페이지 URL 생성 함수
def get_page_url(base_url, page_num):
    return f"{base_url}&p={page_num}"

# 전체 페이지를 크롤링하는 함수
def crawl_all_pages(base_url, start_page, end_page):
    current_page = start_page
    while current_page <= end_page:
        # 페이지 URL 설정
        page_url = get_page_url(base_url, current_page)
        print(f"Moving to page {current_page}: {page_url}")

        # 해당 페이지로 이동
        driver.get(page_url)
        time.sleep(3)  # 페이지 로드 대기

        # 현재 페이지 크롤링
        crawl_current_page()

        current_page += 1

# 기본 URL (q=키워드)
#base_url = "https://section.cafe.naver.com/ca-fe/home/search/articles?q=공동육아&em=1&sm=mtb_opt&ssc=tab.cafe.all&st=rel&stnm=date&opt_tab=0&nso=so%3Add%2Cp%3Afrom20220510to20241002&date_from=20220510&date_to=20241002"
base_url="https://search.naver.com/search.naver?ssc=tab.cafe.all&cafe_where=&query=%EA%B3%B5%EB%8F%99%EC%9C%A1%EC%95%84&ie=utf8&st=date&date_option=8&date_from=2022.10.03&date_to=2022.10.31&srchby=text&dup_remove=1&cafe_url=&without_cafe_url=&sm=tab_opt&nso=so%3Add%2Cp%3Afrom20221003to20221231&nso_open=1&prdtype=0"
# 크롤링할 페이지 범위 설정 (예: 1번 페이지부터 5번 페이지까지만 크롤링)
start_page = 1
end_page = 100

# 크롤링 시작
crawl_all_pages(base_url, start_page, end_page)

# 결과를 CSV 파일로 저장
df = pd.DataFrame({'Title': all_titles, 'Content': all_contents, 'Link': all_links})
df.to_csv("crawled_posts.csv", index=False, encoding='utf-8-sig')
print("Crawling completed and saved to 'crawled_posts.csv'")

# 브라우저 종료
driver.quit()

Moving to page 1: https://search.naver.com/search.naver?ssc=tab.cafe.all&cafe_where=&query=%EA%B3%B5%EB%8F%99%EC%9C%A1%EC%95%84&ie=utf8&st=date&date_option=8&date_from=2022.10.03&date_to=2022.10.31&srchby=text&dup_remove=1&cafe_url=&without_cafe_url=&sm=tab_opt&nso=so%3Add%2Cp%3Afrom20221003to20221231&nso_open=1&prdtype=0&p=1
Found 0 articles on the current page.
Moving to page 2: https://search.naver.com/search.naver?ssc=tab.cafe.all&cafe_where=&query=%EA%B3%B5%EB%8F%99%EC%9C%A1%EC%95%84&ie=utf8&st=date&date_option=8&date_from=2022.10.03&date_to=2022.10.31&srchby=text&dup_remove=1&cafe_url=&without_cafe_url=&sm=tab_opt&nso=so%3Add%2Cp%3Afrom20221003to20221231&nso_open=1&prdtype=0&p=2
Found 0 articles on the current page.
Moving to page 3: https://search.naver.com/search.naver?ssc=tab.cafe.all&cafe_where=&query=%EA%B3%B5%EB%8F%99%EC%9C%A1%EC%95%84&ie=utf8&st=date&date_option=8&date_from=2022.10.03&date_to=2022.10.31&srchby=text&dup_remove=1&cafe_url=&without_cafe_url=&sm=tab_opt&nso=so

KeyboardInterrupt: 

크롤링된 파일의 'Link'주소로 게시글 본문 내용과 카테고리 가져오는 부분입니다.

카테고리는 나중에 내용 분류상 필요할수도 있어서 추가했습니다.

여러번 크롤링 할수록 게시글 차단(카페회원에 한해 공개)이 되어 본문 내용을 가져오지 못하는 경향이 있네요.T.T

본문내용 크롤링 후 'crawled_results.csv'로 저장됩니다


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Initialize the Selenium driver function
def initialize_driver():
    chrome_options = Options()
    chrome_options.binary_location = '/usr/bin/google-chrome'  # Chrome path
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--headless")  # Comment this line if you want to see the browser
    driver = webdriver.Chrome(service=Service('/usr/local/bin/chromedriver'), options=chrome_options)
    return driver

# Start the driver
driver = initialize_driver()
# Log in to Naver
driver.get("https://nid.naver.com/nidlogin.login")
time.sleep(2)
driver.find_element(By.ID, "id").send_keys("postpia")  # Replace with your Naver ID
driver.find_element(By.ID, "pw").send_keys("na700925!shj")  # Replace with your Naver password
driver.find_element(By.CLASS_NAME, "btn_login").click()
time.sleep(5)  # Wait for login to complete
# Load the uploaded CSV file to see the structure and extract link addresses
file_path = '/content/crawled_posts.csv'
df_new_links = pd.read_csv(file_path)
df_new_links['Category']= 'N/A'
# Display the first few rows to understand the structure
df_new_links.head()

# Extracting the URLs from the 'Link' column for the new crawling task
new_urls = df_new_links['Link'].dropna().unique()  # Ensuring unique and non-empty links

# Function to crawl content with enhanced handling for encoding
def extract_content_with_selectors(url):
    try:
        # Initialize the driver and navigate to the URL
        driver.get(url)
        time.sleep(3)  # Wait for the page to load

        # Switch to the main frame containing the content
        driver.switch_to.frame("cafe_main")
        time.sleep(2)

        # Parse the page source with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        # Extract category
        category = soup.select_one("a.link_board")

        if category:
          category = category.get_text(strip=True).split(' ')[0]
        else:
          category = "N/A"

        span_elements = soup.select("span.se-fs-.se-ff-")
        content = "\n".join([element.get_text(strip=True) for element in span_elements])

        # Extract title using a general title tag
        title = soup.title.string.strip() if soup.title else "No Title Found"
        return title, content,category if content else "No Content Found"
    except Exception as e:
        return "Error", str(e)

# Reinitialize the driver for this extraction attempt
driver = initialize_driver()

# Extract content using the updated function
titles_updated = []
contents_updated = []
categories_updated = []
for url in new_urls:
    title, content,category = extract_content_with_selectors(url)
    titles_updated.append(title)
    contents_updated.append(content)
    categories_updated.append(category)
# Close the driver after extraction
driver.quit()

# Display the extracted results in a simple text format
for idx, (url, title, content,category) in enumerate(zip(new_urls, titles_updated, contents_updated,categories_updated), start=1):
    print(f"\n### Post {idx} ###")
    print(f"URL: {url}")
    print(f"Title: {title}")
    print(f"Content:  {content[:100]}...")   # Displaying the first 500 characters of the content for brevity
    print(f"Category: {category}")
# Create a DataFrame from the new data
df_new_content = pd.DataFrame({
    'URL': new_urls,
    'Title': titles_updated,
    'Content': contents_updated,
    'Category': categories_updated
})

# Ensure the URLs match correctly between the existing and new data
df_new_links['Content'] = df_new_links['Link'].map(df_new_content.set_index('URL')['Content'])
df_new_links['Category'] = df_new_links['Link'].map(df_new_content.set_index('URL')['Category'])

# Save the updated DataFrame
df_new_links.to_csv('/content/crawled_results.csv', index=False, encoding='utf-8')

KeyboardInterrupt: 

In [18]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchFrameException, NoSuchElementException
from datetime import datetime, timedelta  # timedelta 추가

# Initialize the Selenium driver function
def initialize_driver():
    chrome_options = Options()
    chrome_options.binary_location = '/usr/bin/google-chrome'  # Chrome path
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--headless")  # Comment this line if you want to see the browser
    driver = webdriver.Chrome(service=Service('/usr/local/bin/chromedriver'), options=chrome_options)
    return driver

# Start the driver
driver = initialize_driver()

# Log in to Naver
driver.get("https://nid.naver.com/nidlogin.login")
time.sleep(2)
driver.find_element(By.ID, "id").send_keys("postpia")  # Replace with your Naver ID
driver.find_element(By.ID, "pw").send_keys("na700925!shj")  # Replace with your Naver password
driver.find_element(By.CLASS_NAME, "btn_login").click()
time.sleep(5)  # Wait for login to complete

# 크롤링한 데이터를 저장할 리스트
cafe_data = []

# 네이버 검색 페이지로 이동
driver.get('https://search.naver.com/search.naver?ssc=tab.cafe.all&cafe_where=&query=%EA%B3%B5%EB%8F%99%EC%9C%A1%EC%95%84&ie=utf8&st=date&date_option=8&date_from=2022.10.03&date_to=2022.10.31&srchby=text&dup_remove=1&cafe_url=&without_cafe_url=&sm=tab_opt&nso=so%3Add%2Cp%3Afrom20221003to20221231&nso_open=1&prdtype=0')

# 페이지 결과 로딩 대기
time.sleep(2)

# 날짜 범위를 나누는 함수 (한 달을 두 번으로 분할)
def get_date_ranges(start_date, end_date):
    date_ranges = []
    current_date = start_date

    while current_date < end_date:
        # 1-15일
        first_half_start = current_date.replace(day=1)
        first_half_end = first_half_start.replace(day=15)
        date_ranges.append((first_half_start, first_half_end))

        # 16일~말일
        second_half_start = first_half_start.replace(day=16)
        second_half_end = (second_half_start + timedelta(days=31)).replace(day=1) - timedelta(days=1)
        date_ranges.append((second_half_start, second_half_end))

        # 다음 달로 이동
        current_date = second_half_start + timedelta(days=31)
        current_date = current_date.replace(day=1)

    return date_ranges

# URL을 날짜에 맞게 생성하는 함수
def build_url(query, start_date, end_date):
    base_url = "https://search.naver.com/search.naver?ssc=tab.cafe.all&cafe_where=&query={}&ie=utf8&st=date".format(query)
    date_range = f"&date_from={start_date.strftime('%Y.%m.%d')}&date_to={end_date.strftime('%Y.%m.%d')}"
    nso = f"&nso=so%3Add%2Cp%3Afrom{start_date.strftime('%Y%m%d')}to{end_date.strftime('%Y%m%d')}&nso_open=1"
    full_url = base_url + date_range + nso
    return full_url

# 스크롤을 사용하여 데이터를 더 많이 로드하는 함수
def scroll_and_crawl(max_scrolls=100):
    last_height = driver.execute_script("return document.body.scrollHeight")
    scrolls = 0

    while scrolls < max_scrolls:
        scrolls += 1
        # 스크롤을 페이지의 가장 아래로 내림
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # 로딩 대기

        # 새로운 높이를 가져옴
        new_height = driver.execute_script("return document.body.scrollHeight")

        # 새로운 게시물이 로드되었는지 확인
        if new_height == last_height:
            print("더 이상 스크롤할 내용이 없습니다.")
            break

        # 페이지에서 데이터 크롤링
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        posts = soup.select('ul.lst_view li.bx')  # 게시글이 있는 영역 선택

        for post in posts:
            # 게시글 제목
            title = post.select_one('a.title_link').text.strip()
            # 게시글 링크
            link = post.select_one('a.title_link')['href']
            # 작성 날짜
            date = post.select_one('div.user_info span.sub').text.strip()

            # 크롤링한 데이터 저장
            cafe_data.append({
                'title': title,
                'link': link,
                'date': date
            })

        # 마지막 높이를 업데이트
        last_height = new_height

# 스크롤을 100번 실행하며 게시글을 크롤링
scroll_and_crawl(max_scrolls=100)

# 시작 날짜와 종료 날짜 설정
start_date = datetime(2022, 5, 10)  # 시작 날짜
end_date = datetime(2022, 6, 30)    # 종료 날짜

# 한 달을 두 번으로 나눈 날짜 범위 생성
date_ranges = get_date_ranges(start_date, end_date)

# 검색어 설정
query = "공동육아"

# 각 날짜 범위에 맞춰 URL을 생성하고 크롤링
for start, end in date_ranges:
    print(f"크롤링 중: {start.strftime('%Y.%m.%d')} - {end.strftime('%Y.%m.%d')}")
    search_url = build_url(query, start, end)

    # 네이버 검색 결과 페이지로 이동
    driver.get(search_url)

    # 페이지 로딩 대기
    time.sleep(2)

    # 스크롤을 통한 크롤링 실행
    scroll_and_crawl(max_scrolls=100)

# 크롤링 결과를 CSV 파일로 저장
with open('crawled_posts.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['title', 'link', 'date']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(cafe_data)

print(f"크롤링 완료. {len(cafe_data)}개의 게시글이 저장되었습니다.")

# 드라이버 종료
driver.quit()
from google.colab import files



# Download the file to your local machine
files.download('crawled_posts.csv')

더 이상 스크롤할 내용이 없습니다.
크롤링 중: 2022.05.01 - 2022.05.15
더 이상 스크롤할 내용이 없습니다.
크롤링 중: 2022.05.16 - 2022.05.31
더 이상 스크롤할 내용이 없습니다.
크롤링 중: 2022.06.01 - 2022.06.15
더 이상 스크롤할 내용이 없습니다.
크롤링 중: 2022.06.16 - 2022.06.30
더 이상 스크롤할 내용이 없습니다.
크롤링 완료. 48828개의 게시글이 저장되었습니다.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>